Esse exércicio, iremos criar um banco de dados sobre vendas utilizando o SQLite e consumir os dados utilizando Pyspark.

In [1]:
# Baixando a biblioteca Pyspark e o driver JDBC do SQLite

! pip install Pyspark

# Download do driver

! wget https://repo1.maven.org/maven2/org/xerial/sqlite-jdbc/3.36.0.3/sqlite-jdbc-3.36.0.3.jar -O /content/sqlite-jdbc.jar

--2025-03-15 11:28:40--  https://repo1.maven.org/maven2/org/xerial/sqlite-jdbc/3.36.0.3/sqlite-jdbc-3.36.0.3.jar
Resolving repo1.maven.org (repo1.maven.org)... 199.232.192.209, 199.232.196.209, 2a04:4e42:4c::209, ...
Connecting to repo1.maven.org (repo1.maven.org)|199.232.192.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9731064 (9.3M) [application/java-archive]
Saving to: ‘/content/sqlite-jdbc.jar’

/content/sqlite-jdb 100%[===================>]   9.28M  --.-KB/s    in 0.1s    

2025-03-15 11:28:41 (85.7 MB/s) - ‘/content/sqlite-jdbc.jar’ saved [9731064/9731064]



In [22]:
# Importando as bibliotecas

import sqlite3
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [23]:
# Criando um banco dedados SQLite localmente

conn = sqlite3.connect('local_database.db')
cursor = conn.cursor()

In [24]:
# Criando uma tabela

cursor.execute('''
  CREATE TABLE IF NOT EXISTS vendas(
    id INTEGER PRIMARY KEY,
    categoria TEXT,
    valor REAL,
    produto TEXT,
    quantidade INTEGER,
    cliente TEXT,
    forma_pagamento TEXT
  )
''')

In [25]:
# Inserindo dados na tabela

cursor.executemany('''
  INSERT INTO vendas (categoria, valor, produto, quantidade, cliente, forma_pagamento)
  VALUES (?, ?, ?, ?, ?, ?)
''',[
    ('eletrônicos', 150.0, 'smartphone', 1, 'João Silva', 'cartão de crédito'),
    ('roupas', 80.0, 'camiseta', 2, 'Maria Oliveira', 'dinheiro'),
    ('eletrônicos', 2000.0, 'notebook', 1, 'Carlos Souza', 'cartão de débito'),
    ('alimentos', 50.0, 'arroz', 10, 'Ana Lima', 'cartão de crédito'),
    ('roupas', 120.0, 'calça jeans', 1, 'Pedro Santos', 'dinheiro'),
    ('eletrônicos', 90.0, 'mouse', 3, 'Lucas Ferreira', 'pix'),
    ('alimentos', 25.0, 'feijão', 5, 'Fernando Costa', 'pix'),
    ('eletrônicos', 300.0, 'tablet', 1, 'Gabriel Almeida', 'dinheiro'),
    ('roupas', 60.0, 'blusa', 2, 'Juliana Rocha', 'cartão de débito'),
    ('alimentos', 30.0, 'macarrão', 4, 'Ricardo Lima', 'cartão de crédito')
])

In [26]:
# Salvando as alterações e fechando a conexão

conn.commit() # Salva as alterações

conn.close() # Fecha a conexão

In [8]:
# Criando a sessão Spark com o driver JDBC

jdbc_driver_path = "/content/sqlite-jdbc.jar" # Caminho do driver

spark = SparkSession.builder.appName("Análise de dados em Banco de dados") \
        .config("spark.driver.extraClassPath", jdbc_driver_path).getOrCreate()

In [27]:
# Carregar os dados do SQLite para um DataFrame do Spark

df = spark.read.format("jdbc").options(
    url = "jdbc:sqlite:local_database.db",
    dbtable = "vendas",
    driver = "org.sqlite.JDBC"
).load()

df.show()

+---+-----------+------+-----------+----------+---------------+-----------------+
| id|  categoria| valor|    produto|quantidade|        cliente|  forma_pagamento|
+---+-----------+------+-----------+----------+---------------+-----------------+
|  1|eletrônicos| 150.0| smartphone|         1|     João Silva|cartão de crédito|
|  2|     roupas|  80.0|   camiseta|         2| Maria Oliveira|         dinheiro|
|  3|eletrônicos|2000.0|   notebook|         1|   Carlos Souza| cartão de débito|
|  4|  alimentos|  50.0|      arroz|        10|       Ana Lima|cartão de crédito|
|  5|     roupas| 120.0|calça jeans|         1|   Pedro Santos|         dinheiro|
|  6|eletrônicos|  90.0|      mouse|         3| Lucas Ferreira|              pix|
|  7|  alimentos|  25.0|     feijão|         5| Fernando Costa|              pix|
|  8|eletrônicos| 300.0|     tablet|         1|Gabriel Almeida|         dinheiro|
|  9|     roupas|  60.0|      blusa|         2|  Juliana Rocha| cartão de débito|
| 10|  alimentos

In [28]:
# Import função coluna do Pyspark
from pyspark.sql.functions import col

# Criando coluna valor total

df = df.withColumn("valor_total", col("valor") * col("quantidade"))

#Exibir o Dataframe atualizado

df.show()

+---+-----------+------+-----------+----------+---------------+-----------------+-----------+
| id|  categoria| valor|    produto|quantidade|        cliente|  forma_pagamento|valor_total|
+---+-----------+------+-----------+----------+---------------+-----------------+-----------+
|  1|eletrônicos| 150.0| smartphone|         1|     João Silva|cartão de crédito|      150.0|
|  2|     roupas|  80.0|   camiseta|         2| Maria Oliveira|         dinheiro|      160.0|
|  3|eletrônicos|2000.0|   notebook|         1|   Carlos Souza| cartão de débito|     2000.0|
|  4|  alimentos|  50.0|      arroz|        10|       Ana Lima|cartão de crédito|      500.0|
|  5|     roupas| 120.0|calça jeans|         1|   Pedro Santos|         dinheiro|      120.0|
|  6|eletrônicos|  90.0|      mouse|         3| Lucas Ferreira|              pix|      270.0|
|  7|  alimentos|  25.0|     feijão|         5| Fernando Costa|              pix|      125.0|
|  8|eletrônicos| 300.0|     tablet|         1|Gabriel Almei

In [45]:
# Agrupamento por Cliente: calcular o total gasto
from pyspark.sql.functions import sum

df_cliente = df.groupBy("cliente").agg(sum("valor_total").alias("total_gasto"))
df_cliente.show()

+---------------+-----------+
|        cliente|total_gasto|
+---------------+-----------+
| Fernando Costa|      125.0|
|Gabriel Almeida|      300.0|
|   Carlos Souza|     2000.0|
|   Pedro Santos|      120.0|
|  Juliana Rocha|      120.0|
| Lucas Ferreira|      270.0|
| Maria Oliveira|      160.0|
|     João Silva|      150.0|
|       Ana Lima|      500.0|
|   Ricardo Lima|      120.0|
+---------------+-----------+



In [30]:
# Análise de Produtos: Encontrar os produtos mais vendidos

df_produtos = df.groupBy("produto").agg(sum("quantidade").alias("quantidade_vendida")) \
              .orderBy(col("quantidade_Vendida").desc())

df_produtos.show()

+-----------+------------------+
|    produto|quantidade_vendida|
+-----------+------------------+
|      arroz|                10|
|     feijão|                 5|
|   macarrão|                 4|
|      mouse|                 3|
|   camiseta|                 2|
|      blusa|                 2|
| smartphone|                 1|
|     tablet|                 1|
|calça jeans|                 1|
|   notebook|                 1|
+-----------+------------------+



In [35]:
# Filtrando o campo forma_pagamento

df_pagamento = df.groupBy("forma_pagamento").count().orderBy("count", ascending = False)
df_pagamento.show()

+-----------------+-----+
|  forma_pagamento|count|
+-----------------+-----+
|         dinheiro|    3|
|cartão de crédito|    3|
| cartão de débito|    2|
|              pix|    2|
+-----------------+-----+



In [36]:
# Transformando o DF em Pandas e Gerando um Excel

import pandas as pd

df_pandas = df_cliente.toPandas()

# Salvando em Excel

df_pandas.to_excel("Clientes.xlsx", index = False)